In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from keras.models import Sequential
from keras.layers import LSTM, Dense
import matplotlib.pyplot as plt
import seaborn as sns
import pygad



In [2]:
# Load data
data = pd.read_csv('processed_Infilteration_2.csv')


In [3]:

# Separate label and timestamp columns
y = data['Label'].values

timestamps = data['Timestamp'].values
X = data.drop(['Label', 'Timestamp'], axis=1).values

# Standardize data
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [4]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the fitness function
def fitness_func(solution, solution_idx):
    # Convert the binary vector to a boolean mask
    feature_mask = np.array(solution, dtype=bool)
    # Select the features using the mask
    X_train_selected = X_train[:, feature_mask]
    X_test_selected = X_test[:, feature_mask]
    # Define the LSTM model
    model = Sequential()
    print(X_train_selected.shape)  # Debugging line
    model.add(LSTM(32, input_shape=(X_train_selected.shape[1],1)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Train the model
    model.fit(X_train_selected, y_train, epochs=10, batch_size=32, verbose=1)
    # Evaluate the model's accuracy
    y_pred = model.predict_classes(X_test_selected)
    cm = confusion_matrix(y_test, y_pred)
    accuracy = np.trace(cm) / np.sum(cm)
    return accuracy

# Define the genetic algorithm parameters
num_generations = 10
num_parents_mating = 4
sol_per_pop = 8
num_features = X_train.shape[1]
mutation_rate = 0.1
crossover_prob = 0.5
# Initialize the population
initial_population = np.random.randint(2, size=(sol_per_pop, num_features)).tolist()

# Create the genetic algorithm object
ga_instance = pygad.GA(initial_population=initial_population,
                       fitness_func=fitness_func,
                       num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_features,
                       mutation_percent_genes=mutation_rate*100,
                       crossover_probability=crossover_prob,
                       suppress_warnings=True)

# Evolve the population
ga_instance.run()

# Get the best solution
solution, solution_fitness, solution_idx = ga_instance.best



(485892, 46)
Epoch 1/10
15185/15185 [==============================] - 244s 16ms/step - loss: 0.3546 - accuracy: 0.8862
Epoch 2/10
 9913/15185 [==================>...........] - ETA: 1:22 - loss: 0.3539 - accuracy: 0.8865

In [ ]:
# Select the features using the best solution
feature_mask = np.array(solution, dtype=bool)
X_train_selected = X_train[:, feature_mask]
X_test_selected = X_test[:, feature_mask]